In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

In [2]:
df = pd.read_excel('Stage2_KWARA_crosschecked.xlsx')
print(df.head(5))

   State  LGA                Ward       PU-Code                      PU-Name  \
0  KWARA  ASA       YOWERE/SOSOKI  23-01-01-002           ONIYEYE OPEN SPACE   
1  KWARA  ASA       YOWERE/SOSOKI  23-01-01-003           LAODU 1 OPEN SPACE   
2  KWARA  ASA       YOWERE/SOSOKI  23-01-01-004          IDI-OSE L.G.E.A SCH   
3  KWARA  ASA  ELEBUE/AGBONA/FATA  23-01-03-004  MEGIDA EDUNJO OPEN SPACE II   
4  KWARA  ASA  ELEBUE/AGBONA/FATA  23-01-03-006         IGBOROKO L.G.E.A SCH   

                                        Full_Address  Latitude  Longitude  \
0               ASA YOWERE/SOSOKI ONIYEYE OPEN SPACE  8.591231   4.399403   
1               ASA YOWERE/SOSOKI LAODU 1 OPEN SPACE  8.581165   4.455840   
2              ASA YOWERE/SOSOKI IDI-OSE L.G.E.A SCH  8.601927   4.407921   
3  ASA ELEBUE/AGBONA/FATA MEGIDA EDUNJO OPEN SPAC...  8.865052   4.070663   
4        ASA ELEBUE/AGBONA/FATA IGBOROKO L.G.E.A SCH  8.264133   4.470320   

   Accredited_Voters  Registered_Voters  ...  Result_She

In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# Convert coordinates to radians
coords = np.radians(df[['Latitude', 'Longitude']].values)

# Build BallTree using Haversine distance
tree = BallTree(coords, metric='haversine')

# Define search radius (convert km to radians)
radius_km = 5
radius = radius_km / 6371.0  # Earth radius in km

# Find neighbours within radius for each point
indices = tree.query_radius(coords, r=radius)

# Store neighbours (excluding self)
df['neighbours'] = [list(neigh[neigh != i]) for i, neigh in enumerate(indices)]

# Add corresponding PU names for neighbours
df['neighbours_pu_name'] = [
    df.loc[neigh, 'PU-Name'].tolist() for neigh in df['neighbours']
]

print("Neighbour search complete!")


Neighbour search complete!


In [4]:
print(df.head(10).to_string(index=False))

State LGA                Ward      PU-Code                     PU-Name                                       Full_Address  Latitude  Longitude  Accredited_Voters  Registered_Voters  Results_Found  Transcription_Count  Result_Sheet_Stamped  Result_Sheet_Corrected  Result_Sheet_Invalid  Result_Sheet_Unclear Result_Sheet_Unsigned  APC  LP  PDP  NNPP                                                                                                       Results_File       neighbours                                                                  neighbours_pu_name
KWARA ASA       YOWERE/SOSOKI 23-01-01-002          ONIYEYE OPEN SPACE               ASA YOWERE/SOSOKI ONIYEYE OPEN SPACE  8.591231   4.399403                220                750           True                   -1                 False                   False                 False                 False               UNKNOWN  106   0  109     0         https://docs.inecelectionresults.net/elections_prod/1292/state/01/lga/01/ward/

In [5]:
# Suppose these are your party columns
parties = ['APC', 'PDP', 'LP', 'NNPP',]

# Initialize columns for outlier scores
for party in parties:
    df[f'{party}_outlier_score'] = np.nan

# Function to compute outlier scores
for i, row in df.iterrows():
    neighbours = df.loc[row['neighbours']]
    for party in parties:
        if len(neighbours) > 0:
            neighbour_mean = neighbours[party].mean()
            neighbour_std = neighbours[party].std()
            if neighbour_std > 0:
                score = abs(row[party] - neighbour_mean) / neighbour_std
                df.loc[i, f'{party}_outlier_score'] = score


In [6]:
print(df.head(10).to_string())

   State  LGA                 Ward       PU-Code                      PU-Name                                        Full_Address   Latitude   Longitude  Accredited_Voters  Registered_Voters  Results_Found  Transcription_Count  Result_Sheet_Stamped  Result_Sheet_Corrected  Result_Sheet_Invalid  Result_Sheet_Unclear Result_Sheet_Unsigned  APC  LP  PDP  NNPP                                                                                                        Results_File        neighbours                                                                   neighbours_pu_name  APC_outlier_score  PDP_outlier_score  LP_outlier_score  NNPP_outlier_score
0  KWARA  ASA        YOWERE/SOSOKI  23-01-01-002           ONIYEYE OPEN SPACE                ASA YOWERE/SOSOKI ONIYEYE OPEN SPACE   8.591231    4.399403                220                750           True                   -1                 False                   False                 False                 False               UNKNOWN  106   

In [9]:
df['neighbours'] = df['neighbours'].apply(lambda x: ', '.join(str(int(i)) for i in x))
df['neighbours_pu_name'] = df['neighbours_pu_name'].apply(lambda x: ', '.join(x))
df.to_csv('cleaned_neighbours.csv', index=False)


In [7]:
import pandas as pd

# Define your CSV file name
output_file = "all_outliers_per_party.csv"

# Create an empty list to store data for all parties
all_parties = []

for party in parties:
    # Sort all rows for the party
    party_outliers = df.sort_values(by=f'{party}_outlier_score', ascending=False).copy()
    party_outliers['party'] = party

    # Convert list columns to readable text
    party_outliers['neighbours'] = party_outliers['neighbours'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
    party_outliers['neighbours_pu_name'] = party_outliers['neighbours_pu_name'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

    # Select relevant columns
    export_cols = ['party', 'PU-Name', f'{party}_outlier_score', 'neighbours', 'neighbours_pu_name']

    # Add to list
    all_parties.append(party_outliers[export_cols])

# Combine all parties into one DataFrame
final_df = pd.concat(all_parties, ignore_index=True)

# Export to CSV (with index)
final_df.to_csv(output_file, index=True)

print("✅ Export complete! All parties saved in one CSV file:", output_file)



✅ Export complete! All parties saved in one CSV file: all_outliers_per_party.csv


In [8]:
# Didnt later use this but keeping for reference
df.to_excel('excel_final_with_outlier_lat_long.xlsx')


In [ ]:
for party in parties:
    top_outliers = df.sort_values(by=f'{party}_outlier_score', ascending=False).head(10)
    print(f"Top outliers for {party}:")
    print(top_outliers[['PU-Name', f'{party}_outlier_score', 'neighbours', 'neighbours_pu_name']].to_string())
    print("\n" + "-"*80 + "\n")


In [78]:
import pandas as pd

# Define your Excel file name
output_file = "all_outliers_per_party.xlsx"

# Create an Excel writer
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for party in parties:
        # Sort all rows for the party
        party_outliers = df.sort_values(by=f'{party}_outlier_score', ascending=False).copy()
        party_outliers['party'] = party

        # Add the original index as a column
        party_outliers.reset_index(inplace=True)
        party_outliers.rename(columns={'index': 'PU_Index'}, inplace=True)

        # Convert lists to readable comma-separated strings
        party_outliers['neighbours'] = party_outliers['neighbours'].apply(
            lambda x: ', '.join(str(int(i)) for i in x) if isinstance(x, (list, tuple)) else str(x)
        )
        party_outliers['neighbours_pu_name'] = party_outliers['neighbours_pu_name'].apply(
            lambda x: ', '.join(map(str, x)) if isinstance(x, (list, tuple)) else str(x)
        )

        # Select relevant columns
        export_cols = ['PU_Index', 'party', 'PU-Name', f'{party}_outlier_score', 'neighbours', 'neighbours_pu_name']

        # Write to new sheet
        party_outliers[export_cols].to_excel(writer, sheet_name=party, index=False)

print("✅ Export complete! Each sheet now includes the PU_Index column showing the original DataFrame index.")


✅ Export complete! Each sheet now includes the PU_Index column showing the original DataFrame index.


In [76]:
import folium

# Create map centered on your state
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=8)

# Get top 10 outliers by APC_outlier_score
top10 = df.sort_values(by='NNPP_outlier_score', ascending=False).head(10)

# Get top 3 separately
top3_indices = top10.head(3).index

# Add the 10 polling units with arrow markers
for i, row in top10.iterrows():
    color = 'red' if i in top3_indices else 'blue'
    icon = folium.Icon(color=color, icon='play', prefix='fa')  # triangle/arrow-like icon
    
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['PU-Name']} (Score: {row['NNPP_outlier_score']})",
        icon=icon
    ).add_to(m)

# Save map
m.save('NNPP_election_outliers_map.html')

print("✅ Map saved — top 3 outliers are red arrows, next 7 are blue arrows.")


✅ Map saved — top 3 outliers are red arrows, next 7 are blue arrows.


In [89]:
import pandas as pd

# Define parties
parties = ['APC', 'PDP', 'LP', 'NNPP']

# Combine all outlier scores into one DataFrame
all_outliers = pd.concat([
    df[['PU-Name', f'{party}_outlier_score', 'Latitude', 'Longitude']]
    .rename(columns={f'{party}_outlier_score': 'Outlier_Score'})
    .assign(Party=party)
    for party in parties
])

# Get top 3 overall outliers regardless of party
top3_all = all_outliers.sort_values(by='Outlier_Score', ascending=False).head(3)

# Reorder columns
top3_all = top3_all[['Party', 'PU-Name', 'Outlier_Score', 'Latitude', 'Longitude']]

# --- Display as styled DataFrame ---
styled_table = (
    top3_all.style
    .background_gradient(subset=['Outlier_Score'], cmap='Reds')
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('border', '1px solid #ccc')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]}
    ])
    .format({'Outlier_Score': '{:.3f}', 'Latitude': '{:.4f}', 'Longitude': '{:.4f}'})
)

styled_table


,Party,PU-Name,Outlier_Score,Latitude,Longitude
769,NNPP,OPEN SPACE ODE OPELE,133.756,8.4347,4.6657
77,APC,WAGURU MARKET,99.702,9.0393,3.6218
1270,PDP,MOLEKE VILLAGE,47.376,8.8310,4.5418


In [90]:
# Save to CSV
top3_all.to_csv('top3_overall_outliers.csv', index=False)

# Save styled table to HTML (for viewing in browser)
styled_table.to_html('top3_overall_outliers.html')
